In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import json
import tensorflowjs as tfjs

## Import Text

In [12]:
with open("./trainingData/sherlockS.txt", 'r', encoding='utf-8') as file:
    text = file.read()

## Tokenize Text

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [14]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

print(input_sequences[:20])

[[268, 61], [268, 61, 19], [268, 61, 19, 93], [268, 61, 19, 93, 14], [268, 61, 19, 93, 14, 94], [268, 61, 19, 93, 14, 94, 1], [268, 61, 19, 93, 14, 94, 1, 95], [268, 61, 19, 93, 14, 94, 1, 95, 6], [18, 145], [18, 145, 96], [18, 145, 96, 62], [18, 145, 96, 62, 269], [18, 145, 96, 62, 269, 97], [18, 145, 96, 62, 269, 97, 146], [18, 145, 96, 62, 269, 97, 146, 147], [98, 148], [98, 148, 10], [98, 148, 10, 7], [98, 148, 10, 7, 63], [98, 148, 10, 7, 63, 93]]


In [15]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

print(input_sequences[:8])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0 268  61]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0 268  61  19]
 [  0   0   0   0   0   0   0   0   0   0   0   0 268  61  19  93]
 [  0   0   0   0   0   0   0   0   0   0   0 268  61  19  93  14]
 [  0   0   0   0   0   0   0   0   0   0 268  61  19  93  14  94]
 [  0   0   0   0   0   0   0   0   0 268  61  19  93  14  94   1]
 [  0   0   0   0   0   0   0   0 268  61  19  93  14  94   1  95]
 [  0   0   0   0   0   0   0 268  61  19  93  14  94   1  95   6]]


In [16]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

print(X[8])
print(y[8])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 18]
145


In [17]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

## Create Model Architecture

In [24]:
model = Sequential()
model.add(Embedding(total_words, 150, input_length=max_sequence_len-1))
model.add(LSTM(300))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 15, 150)           139500    
                                                                 
 lstm_2 (LSTM)               (None, 300)               541200    
                                                                 
 dense_2 (Dense)             (None, 930)               279930    
                                                                 
Total params: 960630 (3.66 MB)
Trainable params: 960630 (3.66 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


## Train Model

In [25]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100


71/71 [==============================] - 4s 35ms/step - loss: 6.3009 - accuracy: 0.0382
Epoch 2/100
71/71 [==============================] - 2s 34ms/step - loss: 5.8172 - accuracy: 0.0382
Epoch 3/100
71/71 [==============================] - 2s 35ms/step - loss: 5.6573 - accuracy: 0.0493
Epoch 4/100
71/71 [==============================] - 2s 34ms/step - loss: 5.5165 - accuracy: 0.0547
Epoch 5/100
71/71 [==============================] - 2s 34ms/step - loss: 5.3565 - accuracy: 0.0644
Epoch 6/100
71/71 [==============================] - 2s 34ms/step - loss: 5.1564 - accuracy: 0.0764
Epoch 7/100
71/71 [==============================] - 2s 34ms/step - loss: 4.9114 - accuracy: 0.0942
Epoch 8/100
71/71 [==============================] - 2s 34ms/step - loss: 4.6189 - accuracy: 0.1204
Epoch 9/100
71/71 [==============================] - 2s 34ms/step - loss: 4.2866 - accuracy: 0.1591
Epoch 10/100
71/71 [==============================] - 2s 34ms/step - loss: 3.9435 - accuracy: 0.1973
Epoch 11/10

## Save Model for JS

In [26]:

output_dir = "./tfjs_models/RNN1" 
tfjs.converters.save_keras_model(model, output_dir)

## Save word index

In [21]:
# wordIndex = {}
# for word in tokenizer.word_index:
#     wordIndex[word] = tokenizer.word_index[word]

# with open('./tokenizer/wordIndex.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(wordIndex, ensure_ascii=False))

## Save Max Sequence Length

In [22]:
# with open('./tokenizer/max_sequence_len.txt', 'w') as f:
#     f.write(str(max_sequence_len))